<a href="https://colab.research.google.com/github/mphipps2/covid-19_open_data_projects/blob/main/covid_Jan6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
project_id = "covid"

from google.colab import auth

import humanize
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery
import pandas as pd

my_project = "bigquery-public-data"
my_dataset = "covid19_jhu_csse"
#my_table = "confirmed_cases"
my_table = "summary"
client = bigquery.Client(project=project_id)



Authenticated


In [ ]:

dataset_ref = client.dataset(my_dataset,project=my_project)

dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))
for table in tables:
        print(table.table_id)

table_ref = dataset_ref.table(my_table)
table = client.get_table(table_ref)
print(table.schema)

df = client.list_rows(table, max_results=5).to_dataframe()
df.head()

confirmed_cases
deaths
recovered_cases
summary
[SchemaField('province_state', 'STRING', 'NULLABLE', None, ()), SchemaField('country_region', 'STRING', 'NULLABLE', None, ()), SchemaField('date', 'DATE', 'NULLABLE', None, ()), SchemaField('latitude', 'FLOAT', 'NULLABLE', None, ()), SchemaField('longitude', 'FLOAT', 'NULLABLE', None, ()), SchemaField('location_geom', 'GEOGRAPHY', 'NULLABLE', None, ()), SchemaField('confirmed', 'INTEGER', 'NULLABLE', None, ()), SchemaField('deaths', 'INTEGER', 'NULLABLE', None, ()), SchemaField('recovered', 'INTEGER', 'NULLABLE', None, ()), SchemaField('active', 'INTEGER', 'NULLABLE', None, ()), SchemaField('fips', 'STRING', 'NULLABLE', None, ()), SchemaField('admin2', 'STRING', 'NULLABLE', None, ()), SchemaField('combined_key', 'STRING', 'NULLABLE', None, ())]


,province_state,country_region,date,latitude,longitude,location_geom,confirmed,deaths,recovered,active,fips,admin2,combined_key
0,Castilla y Leon,Spain,2020-08-05,41.835700,-4.397600,POINT(-4.3976 41.8357),20739,2800,8716,9223,None,None,"Castilla y Leon, Spain"
1,Gilgit-Baltistan,Pakistan,2020-08-05,35.792146,74.982138,POINT(74.982138 35.792146),2218,55,1820,343,None,None,"Gilgit-Baltistan, Pakistan"
2,Nordrhein-Westfalen,Germany,2020-08-05,51.433200,7.661600,POINT(7.6616 51.4332),50068,1751,45000,3317,None,None,"Nordrhein-Westfalen, Germany"
3,Biobio,Chile,2020-11-13,-37.446400,-72.141600,POINT(-72.1416 -37.4464),32338,540,30192,1606,None,None,"Biobio, Chile"
4,Bayern,Germany,2020-11-13,48.790400,11.497900,POINT(11.4979 48.7904),153156,3108,100219,49829,None,None,"Bayern, Germany"


In [ ]:
query = """
SELECT
  country_region, SUM(confirmed) as confirmed_sum, SUM(deaths) as deaths_sum, SUM(recovered) as recovered_sum
FROM
  `bigquery-public-data.covid19_jhu_csse.summary`
WHERE
  date = '2020-10-12'
GROUP BY country_region
ORDER BY confirmed_sum desc;
"""
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)
query_results = client.query(query, job_config=safe_config)
print("query results type: ", type(query_results))
#results = query_results.to_dataframe()
df = pd.DataFrame(query_results)
for col in df.columns:
        print(col)

print("number of results: " , len(df))